In [169]:
import pandas as pd 
import sqlite3
pd.options.display.max_columns = 99
pd.options.display.float_format = '{:,.2f}'.format



In [7]:
data = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1')


In [10]:
for chunk in data:
    chunk.head(5)
chunk.head(5)

,company_permalink,company_name,company_category_code,company_country_code,company_state_code,company_region,company_city,investor_permalink,investor_name,investor_category_code,investor_country_code,investor_state_code,investor_region,investor_city,funding_round_type,funded_at,funded_month,funded_quarter,funded_year,raised_amount_usd
50000,/company/nuorder,NuORDER,fashion,USA,CA,Los Angeles,West Hollywood,/person/mortimer-singer,Mortimer Singer,NaN,NaN,NaN,unknown,NaN,series-a,2012-10-01,2012-10,2012-Q4,2012,3060000.0
50001,/company/chacha,ChaCha,advertising,USA,IN,Indianapolis,Carmel,/person/morton-meyerson,Morton Meyerson,NaN,NaN,NaN,unknown,NaN,series-b,2007-10-01,2007-10,2007-Q4,2007,12000000.0
50002,/company/binfire,Binfire,software,USA,FL,Bocat Raton,Bocat Raton,/person/moshe-ariel,Moshe Ariel,NaN,NaN,NaN,unknown,NaN,angel,2008-04-18,2008-04,2008-Q2,2008,500000.0
50003,/company/binfire,Binfire,software,USA,FL,Bocat Raton,Bocat Raton,/person/moshe-ariel,Moshe Ariel,NaN,NaN,NaN,unknown,NaN,angel,2010-01-01,2010-01,2010-Q1,2010,750000.0
50004,/company/unified-color,Unified Color,software,USA,CA,SF Bay,South San Frnacisco,/person/mr-andrew-oung,Mr. Andrew Oung,NaN,NaN,NaN,unknown,NaN,angel,2010-01-01,2010-01,2010-Q1,2010,NaN


In [11]:
columns = []
for col in chunk.columns:
    columns.append(col)

In [12]:
columns

['company_permalink',
 'company_name',
 'company_category_code',
 'company_country_code',
 'company_state_code',
 'company_region',
 'company_city',
 'investor_permalink',
 'investor_name',
 'investor_category_code',
 'investor_country_code',
 'investor_state_code',
 'investor_region',
 'investor_city',
 'funding_round_type',
 'funded_at',
 'funded_month',
 'funded_quarter',
 'funded_year',
 'raised_amount_usd']

In [102]:
# What data is missing?
data = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1')

missing_data = []

for chunk in data:
    missing_data.append(chunk.isnull().sum())

combined_missing_data = pd.concat(missing_data)
unique_combined_mv_vc = combined_missing_data.groupby(combined_missing_data.index).sum()
unique_combined_mv_vc.sort_values(ascending=False)


investor_category_code    50427
investor_state_code       16809
investor_city             12480
investor_country_code     12001
raised_amount_usd          3599
company_category_code       643
company_city                533
company_state_code          492
funding_round_type            3
funded_year                   3
funded_month                  3
funded_at                     3
funded_quarter                3
investor_name                 2
investor_permalink            2
investor_region               2
company_region                1
company_permalink             1
company_name                  1
company_country_code          1
dtype: int64

In [103]:
# Let's drop columns not useful to our analysis
drop_columns = ["investor_permalink",
                "company_permalink",
                "investor_category_code",
               "investor_category_code",
                "investor_state_code",
                "investor_city",
                "investor_country_code",
               ]

data_keep_columns = chunk.columns.drop(drop_columns)

In [104]:
data_keep_columns = data_keep_columns.tolist()

In [105]:
data_keep_columns

['company_name',
 'company_category_code',
 'company_country_code',
 'company_state_code',
 'company_region',
 'company_city',
 'investor_name',
 'investor_region',
 'funding_round_type',
 'funded_at',
 'funded_month',
 'funded_quarter',
 'funded_year',
 'raised_amount_usd']

In [106]:
data = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1', usecols=data_keep_columns)


In [107]:
chunk.head(5)

,company_permalink,company_name,company_category_code,company_country_code,company_state_code,company_region,company_city,investor_permalink,investor_name,investor_category_code,investor_country_code,investor_state_code,investor_region,investor_city,funding_round_type,funded_at,funded_month,funded_quarter,funded_year,raised_amount_usd
50000,/company/nuorder,NuORDER,fashion,USA,CA,Los Angeles,West Hollywood,/person/mortimer-singer,Mortimer Singer,NaN,NaN,NaN,unknown,NaN,series-a,2012-10-01,2012-10,2012-Q4,2012,3060000.0
50001,/company/chacha,ChaCha,advertising,USA,IN,Indianapolis,Carmel,/person/morton-meyerson,Morton Meyerson,NaN,NaN,NaN,unknown,NaN,series-b,2007-10-01,2007-10,2007-Q4,2007,12000000.0
50002,/company/binfire,Binfire,software,USA,FL,Bocat Raton,Bocat Raton,/person/moshe-ariel,Moshe Ariel,NaN,NaN,NaN,unknown,NaN,angel,2008-04-18,2008-04,2008-Q2,2008,500000.0
50003,/company/binfire,Binfire,software,USA,FL,Bocat Raton,Bocat Raton,/person/moshe-ariel,Moshe Ariel,NaN,NaN,NaN,unknown,NaN,angel,2010-01-01,2010-01,2010-Q1,2010,750000.0
50004,/company/unified-color,Unified Color,software,USA,CA,SF Bay,South San Frnacisco,/person/mr-andrew-oung,Mr. Andrew Oung,NaN,NaN,NaN,unknown,NaN,angel,2010-01-01,2010-01,2010-Q1,2010,NaN


In [108]:
column_types = {}

for chunk in data:
    for column in chunk.columns:
        if column not in column_types:
            column_types[column] = [str(chunk.dtypes[column])]
        else:
            column_types[column].append(str(chunk.dtypes[column]))
            


In [109]:
column_types

{'company_category_code': ['object',
  'object',
  'object',
  'object',
  'object',
  'object',
  'object',
  'object',
  'object',
  'object',
  'object'],
 'company_city': ['object',
  'object',
  'object',
  'object',
  'object',
  'object',
  'object',
  'object',
  'object',
  'object',
  'object'],
 'company_country_code': ['object',
  'object',
  'object',
  'object',
  'object',
  'object',
  'object',
  'object',
  'object',
  'object',
  'object'],
 'company_name': ['object',
  'object',
  'object',
  'object',
  'object',
  'object',
  'object',
  'object',
  'object',
  'object',
  'object'],
 'company_region': ['object',
  'object',
  'object',
  'object',
  'object',
  'object',
  'object',
  'object',
  'object',
  'object',
  'object'],
 'company_state_code': ['object',
  'object',
  'object',
  'object',
  'object',
  'object',
  'object',
  'object',
  'object',
  'object',
  'object'],
 'funded_at': ['object',
  'object',
  'object',
  'object',
  'object',
  'objec

In [110]:
unique_column_types = {}

for key, value in column_types.items():
    unique_column_types[key] = set(column_types[key])

In [111]:
unique_column_types

{'company_category_code': {'object'},
 'company_city': {'object'},
 'company_country_code': {'object'},
 'company_name': {'object'},
 'company_region': {'object'},
 'company_state_code': {'object'},
 'funded_at': {'object'},
 'funded_month': {'object'},
 'funded_quarter': {'object'},
 'funded_year': {'float64', 'int64'},
 'funding_round_type': {'object'},
 'investor_name': {'object'},
 'investor_region': {'object'},
 'raised_amount_usd': {'float64'}}

In [112]:
date_columns = ["funded_at", "funded_month", "funded_quarter", "funded_year"]

In [114]:
chunk.head(5)

,company_name,company_category_code,company_country_code,company_state_code,company_region,company_city,investor_name,investor_region,funding_round_type,funded_at,funded_month,funded_quarter,funded_year,raised_amount_usd
50000,NuORDER,fashion,USA,CA,Los Angeles,West Hollywood,Mortimer Singer,unknown,series-a,2012-10-01,2012-10,2012-Q4,2012,3060000.0
50001,ChaCha,advertising,USA,IN,Indianapolis,Carmel,Morton Meyerson,unknown,series-b,2007-10-01,2007-10,2007-Q4,2007,12000000.0
50002,Binfire,software,USA,FL,Bocat Raton,Bocat Raton,Moshe Ariel,unknown,angel,2008-04-18,2008-04,2008-Q2,2008,500000.0
50003,Binfire,software,USA,FL,Bocat Raton,Bocat Raton,Moshe Ariel,unknown,angel,2010-01-01,2010-01,2010-Q1,2010,750000.0
50004,Unified Color,software,USA,CA,SF Bay,South San Frnacisco,Mr. Andrew Oung,unknown,angel,2010-01-01,2010-01,2010-Q1,2010,NaN


In [115]:
data = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1', usecols=data_keep_columns, parse_dates=date_columns)


In [116]:
total_mem = chunk.memory_usage(deep=True)

total_mem_sum = total_mem.sum()

total_mem_used = total_mem_sum / (1024*1024)

print(total_mem_used)


2.1582107543945312


## Results ##

* After dropping unnecessary columns and converting various date fields to date type, total memory usage is under 3 MB.

## Import data into sqlite3 database ## 

In [118]:
conn = sqlite3.connect('crunchbase.db')

In [121]:
for chunk in data:
    chunk.to_sql("investments", conn, if_exists="append", index=False)

In [126]:
query = "SELECT * FROM investments LIMIT 10"

pd.read_sql(query, conn)



,company_name,company_category_code,company_country_code,company_state_code,company_region,company_city,investor_name,investor_region,funding_round_type,funded_at,funded_month,funded_quarter,funded_year,raised_amount_usd
0,Visible World,advertising,USA,NY,New York,New York,AllianceBernstein,New York,series-c+,2008-04-14 00:00:00,2008-04-01 00:00:00,2008-04-01 00:00:00,2008-01-01 00:00:00,25000000.0
1,obopay,mobile,USA,CA,SF Bay,Redwood City,AllianceBernstein,New York,series-c+,2008-04-21 00:00:00,2008-04-01 00:00:00,2008-04-01 00:00:00,2008-01-01 00:00:00,20000000.0
2,Jumptap,mobile,USA,MA,Boston,Boston,AllianceBernstein,New York,series-c+,2008-08-26 00:00:00,2008-08-01 00:00:00,2008-07-01 00:00:00,2008-01-01 00:00:00,26000000.0
3,Avid Radiopharmaceuticals,biotech,USA,PA,Philadelphia,Philadelphia,AllianceBernstein,New York,series-c+,2009-05-21 00:00:00,2009-05-01 00:00:00,2009-04-01 00:00:00,2009-01-01 00:00:00,34500000.0
4,Avid Radiopharmaceuticals,biotech,USA,PA,Philadelphia,Philadelphia,AllianceBernstein,New York,series-c+,2007-05-09 00:00:00,2007-05-01 00:00:00,2007-04-01 00:00:00,2007-01-01 00:00:00,26000000.0
5,Pacific Biosciences,biotech,USA,CA,SF Bay,Menlo Park,AllianceBernstein,New York,series-c+,2009-10-01 00:00:00,2009-10-01 00:00:00,2009-10-01 00:00:00,2009-01-01 00:00:00,68000000.0
6,Fluidigm,biotech,USA,CA,SF Bay,South San Francisco,AllianceBernstein,New York,series-c+,2007-01-10 00:00:00,2007-01-01 00:00:00,2007-01-01 00:00:00,2007-01-01 00:00:00,17000000.0
7,Metabolex,biotech,USA,CA,SF Bay,Hayward,AllianceBernstein,New York,series-c+,2007-04-19 00:00:00,2007-04-01 00:00:00,2007-04-01 00:00:00,2007-01-01 00:00:00,32000000.0
8,Impinj,semiconductor,USA,WA,Seattle,Seattle,AllianceBernstein,New York,series-c+,2007-03-07 00:00:00,2007-03-01 00:00:00,2007-01-01 00:00:00,2007-01-01 00:00:00,19000000.0
9,Targeted Growth,biotech,USA,WA,Seattle,Seattle,AllianceBernstein,New York,venture,2007-02-07 00:00:00,2007-02-01 00:00:00,2007-01-01 00:00:00,2007-01-01 00:00:00,22300000.0



## Analysis ## 
Now that we have our data set up in sqlite3 - let's look at the following questions:

* What proportion of the total amount of funds did the top 10% raise? What about the top 1%? Compare these values to the proportions the bottom 10% and bottom 1% raised.

* Which category of company attracted the most investments?

* Which investor contributed the most money (across all startups)?

* Which investors contributed the most money per startup?

* Which funding round was the most popular? Which was the least popular?


In [176]:
query = """
           SELECT
           CAST(SUM(raised_amount_usd) AS INT64) AS total_funding
           FROM investments
           ORDER BY total_funding DESC
        """

pd.read_sql(query, conn)

,total_funding
0,592481886704


In [177]:
query = """
           SELECT company_name,
           CAST(SUM(raised_amount_usd) AS INT64) AS total_raised
           FROM investments
           GROUP BY company_name
           ORDER BY total_raised DESC
        """

pd.read_sql(query, conn)

,company_name,total_raised
0,Groupon,"10,020,400,000.00"
1,Facebook,"3,901,400,000.00"
2,Nanosolar,"3,760,000,000.00"
3,SurveyMonkey,"2,800,000,000.00"
4,sigmacare,"2,600,000,000.00"
5,Zynga,"2,586,013,000.00"
6,Dropbox,"2,508,400,000.00"
7,Clearwire,"2,420,000,000.00"
8,Bloom Energy,"2,380,000,000.00"
9,Fab.com,"2,226,325,000.00"


In [130]:
query = """
           SELECT company_category_code,
           SUM(raised_amount_usd) AS total_raised
           FROM investments
           GROUP BY company_category_code
           ORDER BY total_raised DESC
        """

pd.read_sql(query, conn)

,company_category_code,total_raised
0,biotech,1.007142e+11
1,software,6.659789e+10
2,cleantech,4.746192e+10
3,enterprise,4.192625e+10
4,web,3.616175e+10
5,mobile,3.360948e+10
6,medical,2.347634e+10
7,advertising,2.214924e+10
8,network_hosting,2.098262e+10
9,semiconductor,2.026810e+10


In [133]:
query = """
           SELECT investor_name,
           SUM(raised_amount_usd) AS total_invested
           FROM investments
           GROUP BY investor_name
           ORDER BY total_invested DESC
        """

pd.read_sql(query, conn)

,investor_name,total_invested
0,Kleiner Perkins Caufield & Byers,1.121783e+10
1,New Enterprise Associates,9.692542e+09
2,Goldman Sachs,6.375459e+09
3,Sequoia Capital,6.039402e+09
4,Greylock Partners,4.960983e+09
5,Intel Capital,4.695617e+09
6,Draper Fisher Jurvetson (DFJ),4.501461e+09
7,Oak Investment Partners,4.350065e+09
8,Andreessen Horowitz,4.233674e+09
9,Lightspeed Venture Partners,4.140979e+09


In [134]:
query = """
           SELECT investor_name,
           AVG(raised_amount_usd) AS avg_invested
           FROM investments
           GROUP BY investor_name
           ORDER BY avg_invested DESC
        """

pd.read_sql(query, conn)

,investor_name,avg_invested
0,Marlin Equity Partners,2.600000e+09
1,GI Partners,1.050000e+09
2,Eagle River Holdings,6.142500e+08
3,Laurel Crown Partners,4.500000e+08
4,Digital Sky Technologies,3.652259e+08
5,Apple Ventures,3.190000e+08
6,Clairvoyant Capital Services,3.190000e+08
7,Crystal Ball Ventures,3.190000e+08
8,NASA Bioventures,3.190000e+08
9,Omniscient Venture Partners,3.190000e+08


In [136]:
query = """
           SELECT funding_round_type,
           COUNT(*) AS number_of_investments
           FROM investments
           GROUP BY funding_round_type
           ORDER BY number_of_investments DESC
        """

pd.read_sql(query, conn)

,funding_round_type,number_of_investments
0,series-a,12975
1,series-c+,9966
2,venture,8113
3,series-b,8092
4,angel,7555
5,other,827
6,private-equity,319
7,post-ipo,16
8,crowdfunding,4
9,None,3


## Results ## 
Based on our queries above we can answer the questions below:

* What proportion of the total amount of funds did the top 10% raise? What about the top 1%? Compare these values to the proportions the bottom 10% and bottom 1% raised.
  ** TBD **

* Which category of company attracted the most investments?
  ** Biotech companies attracted the most investment**

* Which investor contributed the most money (across all startups)?
  ** Kleiner Perkins Caufield & Byers contributed the most **
* Which investors contributed the most money per startup?
  ** On average, Marlin Equity Partners	contributed the most **

* Which funding round was the most popular? Which was the least popular?
  ** Series A was the most popular. While Post-IPO and Crowdfunding were the least popular **